In [1]:
import os
import sys
import json
from pathlib import Path
sys.path.append(os.path.abspath('..'))

# ----------- local imports ----------- 
from constants import FACE_ID_TRAIN_PATH, DATA_DIR
from Facenet.face_id_dataset import get_train_val_set, get_embedding_from_path

import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from PIL import Image
import pandas as pd
import numpy as np


from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.svm import SVC, OneClassSVM
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score


2025-03-20 22:24:52.253919: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-20 22:24:52.429955: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-20 22:24:52.481822: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-20 22:24:52.924402: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-20 22:24:55.520338: W tensorflow/compiler/tf2

In [2]:
# preprocess train embedding dataframe
df_train_embeddings = get_embedding_from_path("embeddings/train_embeddings.csv")

train_df, val_df = get_train_val_set(df_train_embeddings, unknown_cnt = 7)

In [3]:
len(train_df['person'].unique()), len(val_df['person'].unique())

(118, 125)

In [4]:
X = np.vstack(train_df['embeddings'].values) 
y = train_df['person'].values 

X_val = np.vstack(val_df['embeddings'].values)
y_val = val_df['gt'].values  # Ground truth (including "doesn't_exist")


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [5]:
# X.shape
X_norm  = (X / np.linalg.norm(X, axis=1).reshape(-1, 1))
X_val_norm = (X_val / np.linalg.norm(X_val, axis=1).reshape(-1, 1))

In [10]:
# svc_param_grid = {
#     'C': [0.1, 1, 10],  # Regularization strength
#     'kernel': ['linear', 'rbf'],  # SVC kernels
# }

# svc = SVC(probability=True)
# svc_grid = GridSearchCV(svc, svc_param_grid, cv=3, scoring='accuracy', verbose=1)
# svc_grid.fit(X_norm, y_encoded)

# best_svc = svc_grid.best_estimator_
# print(f"Best SVC Parameters: {svc_grid.best_params_}")

best_svc = SVC(kernel='rbf', C=10, probability=True)
best_svc.fit(X_norm, y_encoded)

SVC(C=10, probability=True)

In [13]:
# best_oc_svm = None
# best_nu = None
# best_accuracy = 0

# for nu in [0.05, 0.1, 0.2]:  # Tune nu (outlier sensitivity)
#     oc_svm = OneClassSVM(nu=nu, kernel="rbf")
#     oc_svm.fit(X_norm)  # Train on normal class only

#     oc_predictions = oc_svm.predict(X_val_norm)  # -1 means "doesn't_exist", 1 means "exists"

#     # Compute accuracy by treating "doesn't_exist" as -1 and others as +1
#     oc_labels = np.array([1 if p != "doesn't_exist" else -1 for p in y_val])
#     acc = accuracy_score(oc_labels, oc_predictions)

#     if acc > best_accuracy:
#         best_accuracy = acc
#         best_oc_svm = oc_svm
#         best_nu = nu

best_oc_svm = OneClassSVM(nu=0.05, kernel="rbf")
best_oc_svm.fit(X_norm)

OneClassSVM(nu=0.05)

In [100]:
best_nu, best_accuracy

(0.05, 0.82453909726637)

# Validate

In [14]:
predicted_labels = best_svc.predict(X_val_norm)
predicted_persons = label_encoder.inverse_transform(predicted_labels)
oc_predictions = best_oc_svm.predict(X_val_norm) 

final_predictions = [
    "doesn't_exist" if oc_pred == -1 else svc_pred
    for oc_pred, svc_pred in zip(oc_predictions, predicted_persons)
]


accuracy = accuracy_score(y_val, final_predictions)
print(f"Validation Accuracy using OC-SVM: {accuracy:.4f}")

Validation Accuracy using OC-SVM: 0.8163


In [15]:
prob = best_svc.predict_proba(X_val_norm)
max_prob_values = np.max(prob, axis=1)  # Rename to avoid shadowing

predicted_labels = best_svc.predict(X_val_norm)
predicted_persons = label_encoder.inverse_transform(predicted_labels)

final_predictions = [
    "doesn't_exist" if prob_val < 0.2 else svc_pred
    for prob_val, svc_pred in zip(max_prob_values, predicted_persons)
]

accuracy = accuracy_score(y_val, final_predictions)
print(f"Validation Accuracy using SVM with Th: {accuracy:.4f}")


Validation Accuracy using SVM with Th: 0.9015


In [16]:
similarities = np.dot(X_norm, X_val_norm.T)
best_sim = similarities.max(axis= 0)
best_sim_id = similarities.argmax(axis= 0)
best_sim_p = train_df.iloc[best_sim_id]['person'].values

In [17]:
final_predictions = [
    "doesn't_exist" if sim < 0.6 else svc_pred
    for sim, svc_pred in zip(best_sim, predicted_persons)
]

accuracy = accuracy_score(y_val, final_predictions)
print(f"Validation Accuracy using SVM with Cosine Sim Th: {accuracy:.4f}")

Validation Accuracy using SVM with Cosine Sim Th: 0.9358


In [18]:
wrong_mask = y_val != final_predictions
wrong_mask.sum()

101

In [27]:
val_df_ = val_df.copy()
val_df_['pred'] = final_predictions

# val_df_[wrong_mask]['gt'].value_counts()

In [28]:
# val_df_[wrong_mask]['pred'].value_counts()

# Test

In [25]:
def get_test_embedding_from_path(embedding_path = "embeddings/test_embeddings.csv"):
    """ preprocess train embedding dataframe """
    # preprocess train embedding dataframe
    df_test_embeddings = pd.read_csv(embedding_path)
    # fix embeddings to np array
    df_test_embeddings['embeddings'] = df_test_embeddings['embeddings'].apply(lambda x: np.array(x[1:-1].split(), dtype=np.float32))
    
    df_test_embeddings['img'] = df_test_embeddings['identity'].apply(lambda x : Path(x).name)
    df_test_embeddings['img_no'] = df_test_embeddings['img'].apply(lambda x : int(x.split('.')[0]))
    
    return df_test_embeddings.sort_values(by='img_no')
df_test_embeddings = get_test_embedding_from_path("embeddings/test_embeddings.csv")

In [22]:
X_all = np.vstack(df_train_embeddings['embeddings'].values) 
X_all_norm  = (X_all / np.linalg.norm(X_all, axis=1).reshape(-1, 1))

y_all = df_train_embeddings['person'].values 

label_encoder = LabelEncoder()
y_all_encoded = label_encoder.fit_transform(y_all)

In [24]:
svc = SVC(kernel='rbf', C=10)
oc_svm = OneClassSVM(nu=0.05, kernel="rbf")

svc.fit(X_all_norm, y_all_encoded)
oc_svm.fit(X_all_norm)

OneClassSVM(nu=0.05)

In [29]:
X_test = np.vstack(df_test_embeddings['embeddings'].values)
X_test_norm  = (X_test / np.linalg.norm(X_test, axis=1).reshape(-1, 1))

predicted_labels = best_svc.predict(X_test_norm)
predicted_persons = label_encoder.inverse_transform(predicted_labels)

similarities = np.dot(X_all_norm, X_test_norm.T)
best_sim = similarities.max(axis= 0)
best_sim_id = similarities.argmax(axis= 0)
best_sim_p = df_train_embeddings.iloc[best_sim_id]['person'].values

final_predictions = [
    "doesn't_exist" if sim < 0.6 else svc_pred
    for sim, svc_pred in zip(best_sim, predicted_persons)
]

In [33]:
# final_predictions[-10:]

In [34]:
predications = {k:v for k, v in zip(df_test_embeddings['img'].values, final_predictions)}

In [55]:
import ast 

df_submission = pd.read_csv('../../submissions/_tracking_crowd_mot.csv')

df_submission['objects'] = df_submission['objects'].apply(lambda x: ast.literal_eval(x))

assert isinstance(df_submission['objects'].iloc[0], list), "The first 'objects' entry is not a list!"

In [57]:
df_submission[mask]['objects'].apply(lambda x : x['gt']).value_counts().sort_values()

objects
person_60           2
person_48           2
person_27           3
person_122          3
person_10           3
                 ... 
person_107         79
person_19          83
person_117        100
person_90         114
doesn't_exist    1921
Name: count, Length: 126, dtype: int64

In [52]:
mask = df_submission['objective'] == "face_reid"
df_submission.loc[mask, 'objects'] = df_submission.loc[mask, 'objects'].apply(
    lambda x: {'gt': predications[x['image'].split('/')[1]], 'image': x['image']}
)

In [54]:
df_submission[mask]['objects'].apply(lambda x : x['gt']).value_counts().sort_values()

objects
person_10           2
person_12           3
person_44           3
person_36           4
person_14           5
                 ... 
person_82         104
person_58         106
person_115        114
person_84         188
doesn't_exist    1169
Name: count, Length: 119, dtype: int64

In [44]:
df_submission.to_csv('../../submissions/_face_reid_svc_cosine.csv')

In [45]:
final_predictions.count("doesn't_exist")

1169

In [ ]:
# final_predictions.count()

TypeError: list.count() takes exactly one argument (0 given)